In [1]:
# Cell 1: 配置（修改这里切换模型）
CONFIG_PATH = "../configs/scheme_d.yaml"
CHECKPOINT_PATH = "../checkpoints/scheme_d/best_model.pt"
N_SAMPLES = 6


In [ ]:
# Cell 0: 独立PPG分析（不需要加载模型，可以直接运行这个cell）
# 分析视频RGB信号是否包含有效的心跳节律信息

%matplotlib inline
import os
import numpy as np
import pandas as pd
import cv2
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt
import json

# 找到项目根目录
_cwd = os.getcwd()
if os.path.exists(os.path.join(_cwd, "training_data/samples")):
    ROOT = _cwd
elif os.path.exists(os.path.join(_cwd, "../training_data/samples")):
    ROOT = os.path.dirname(_cwd)
else:
    ROOT = _cwd
os.chdir(ROOT)
print(f"Project root: {ROOT}")

def get_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    info = {
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    }
    info["file_size_mb"] = os.path.getsize(video_path) / (1024 * 1024)
    info["duration_s"] = info["frame_count"] / info["fps"] if info["fps"] > 0 else 0
    info["bitrate_kbps"] = (info["file_size_mb"] * 8 * 1024) / info["duration_s"] if info["duration_s"] > 0 else 0
    cap.release()
    return info

def extract_rgb_signal(video_path, max_frames=None):
    cap = cv2.VideoCapture(video_path)
    if max_frames is None:
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    rgb_signals = []
    for _ in range(max_frames):
        ret, frame = cap.read()
        if not ret: break
        rgb_signals.append([frame[:,:,2].mean(), frame[:,:,1].mean(), frame[:,:,0].mean()])
    cap.release()
    return np.array(rgb_signals)

def bandpass_filter(sig, fs, lowcut=0.7, highcut=4.0, order=3):
    nyq = 0.5 * fs
    b, a = scipy_signal.butter(order, [lowcut/nyq, highcut/nyq], btype='band')
    return scipy_signal.filtfilt(b, a, sig)

def get_dominant_hr(sig, fs):
    sig = sig - sig.mean()
    freqs = np.fft.rfftfreq(len(sig), 1/fs)
    fft_mag = np.abs(np.fft.rfft(sig))
    hr_mask = (freqs >= 0.7) & (freqs <= 3.5)
    if not hr_mask.any(): return None, 0, freqs, fft_mag
    hr_freqs, hr_mags = freqs[hr_mask], fft_mag[hr_mask]
    peak_idx = np.argmax(hr_mags)
    snr = hr_mags[peak_idx]**2 / (hr_mags**2).sum()
    return hr_freqs[peak_idx] * 60, snr, freqs, fft_mag

# 分析
samples_dir = "training_data/samples"
pairs = sorted([d for d in os.listdir(samples_dir) if d.startswith("pair_")])[:8]

print("="*70)
print("视频PPG信号分析 - 检查视频压缩是否影响心跳节律提取")
print("="*70)

fig, axes = plt.subplots(len(pairs), 4, figsize=(16, 2.5*len(pairs)))

for idx, pair in enumerate(pairs):
    pair_dir = os.path.join(samples_dir, pair)
    video_path = os.path.join(pair_dir, "video_0.mp4")
    ecg_path = os.path.join(pair_dir, "ecg.csv")
    meta_path = os.path.join(pair_dir, "metadata.json")
    
    vinfo = get_video_info(video_path)
    fps = vinfo['fps']
    
    # 提取前10秒RGB信号
    rgb = extract_rgb_signal(video_path, min(int(10 * fps), vinfo['frame_count']))
    t = np.arange(len(rgb)) / fps
    
    # 带通滤波红色通道
    red_norm = (rgb[:,0] - rgb[:,0].mean()) / (rgb[:,0].std() + 1e-8)
    red_filt = bandpass_filter(red_norm, fps)
    
    # FFT检测心率
    ppg_hr, snr, freqs, fft_mag = get_dominant_hr(red_filt, fps)
    
    # 读取标注心率
    with open(meta_path) as f:
        annotated_hr = json.load(f).get("heart_rate")
    
    # 打印结果
    if ppg_hr:
        err = abs(ppg_hr - annotated_hr)
        status = "✓好" if err < 10 else ("△一般" if err < 20 else "✗差")
        print(f"{pair}: {vinfo['bitrate_kbps']:.0f}kbps | PPG={ppg_hr:.0f} GT={annotated_hr} 误差={err:.0f} {status} | SNR={snr:.3f}")
    else:
        print(f"{pair}: {vinfo['bitrate_kbps']:.0f}kbps | 未检测到PPG | GT={annotated_hr}")
    
    # 绘图
    ax = axes[idx]
    ax[0].plot(t, rgb[:,0], 'r-', lw=0.5); ax[0].set_ylabel(pair[-4:]); ax[0].grid(True, alpha=0.3)
    ax[1].plot(t, red_filt, 'r-', lw=0.5); ax[1].grid(True, alpha=0.3)
    hr_mask = (freqs >= 0.5) & (freqs <= 4)
    ax[2].plot(freqs[hr_mask]*60, fft_mag[hr_mask], 'b-')
    if ppg_hr: ax[2].axvline(ppg_hr, color='r', ls='--', alpha=0.7)
    if annotated_hr: ax[2].axvline(annotated_hr, color='g', ls=':', alpha=0.7)
    ax[2].set_xlim(40, 150); ax[2].grid(True, alpha=0.3)
    ecg = pd.read_csv(ecg_path)["ecg_counts_filt_monitor"].values[:2500]
    ax[3].plot(np.arange(len(ecg))/250, (ecg-ecg.mean())/(ecg.std()+1e-8), 'b-', lw=0.5); ax[3].grid(True, alpha=0.3)
    
    if idx == 0:
        ax[0].set_title('Red Raw'); ax[1].set_title('Red Filtered'); ax[2].set_title('FFT (BPM)'); ax[3].set_title('ECG')
    if idx == len(pairs)-1:
        for a in ax: a.set_xlabel('Time (s)')

plt.tight_layout()
os.makedirs("eval_results/visualize", exist_ok=True)
plt.savefig("eval_results/visualize/ppg_quality_check.png", dpi=150, bbox_inches='tight')
print(f"\n图片已保存: eval_results/visualize/ppg_quality_check.png")
plt.show()

print("\n" + "="*70)
print("结论：如果大部分样本PPG心率与GT心率误差<10BPM，说明视频包含有效PPG信息")
print("如果误差普遍>20BPM或SNR<0.05，说明视频压缩可能导致PPG信息丢失")
print("="*70)

In [ ]:
# Cell 2: 导入和初始化
%matplotlib inline
import os
import numpy as np
import pandas as pd
import torch
import yaml
import matplotlib.pyplot as plt

from dataset import create_datasets
from video_ecg_model import build_model

# 不依赖 cwd：在多个可能的工作目录下查找 config 文件，确定项目根目录
_cwd = os.getcwd()
_config_name = os.path.basename(CONFIG_PATH)  # e.g. scheme_a.yaml
_candidates = [
    os.path.join(_cwd, CONFIG_PATH),
    os.path.join(_cwd, "models", CONFIG_PATH),
    os.path.join(_cwd, "ECG", "configs", _config_name),
    os.path.join(_cwd, "configs", _config_name),
]
CONFIG_PATH_ABS = None
for p in _candidates:
    p = os.path.abspath(p)
    if os.path.isfile(p):
        CONFIG_PATH_ABS = p
        break
if CONFIG_PATH_ABS is None:
    raise FileNotFoundError(f"Config not found. cwd={_cwd!r}, CONFIG_PATH={CONFIG_PATH!r}. Tried: {_candidates}")
# 项目根 = config 所在目录的上一级（configs/ 的父目录）
_ROOT = os.path.dirname(os.path.dirname(CONFIG_PATH_ABS))
os.chdir(_ROOT)
# checkpoint 相对项目根（去掉开头的 ../ 或 ./，避免 join(ROOT, "../checkpoints/...") 跑到 ROOT 上一级）
_checkpoint_rel = CHECKPOINT_PATH.lstrip("./").lstrip("/")
CHECKPOINT_PATH_ABS = os.path.normpath(os.path.join(_ROOT, _checkpoint_rel))
print(f"Project root: {_ROOT}")

# 提取 scheme 名称用于区分保存路径
SCHEME_NAME = os.path.splitext(_config_name)[0]  # e.g. "scheme_d"
SAVE_DIR = f"eval_results/visualize/{SCHEME_NAME}"
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Scheme: {SCHEME_NAME}, Save dir: {SAVE_DIR}")

# 加载配置
with open(CONFIG_PATH_ABS) as f:
    cfg = yaml.safe_load(f)

# 设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# 加载模型
model = build_model(cfg).to(device)
ckpt = torch.load(CHECKPOINT_PATH_ABS, map_location=device)
# 兼容两种格式：纯 state_dict 或 {'model_state_dict': state_dict}
state_dict = ckpt["model_state_dict"] if isinstance(ckpt, dict) and "model_state_dict" in ckpt else ckpt
model.load_state_dict(state_dict)
model.eval()
print(f"Loaded: {CHECKPOINT_PATH_ABS}")

# 数据
use_imu = cfg.get("data", {}).get("use_imu", False)
_, _, test_ds = create_datasets(cfg)
test_loader = torch.utils.data.DataLoader(
    test_ds,
    batch_size=cfg["train"]["batch_size"],
    shuffle=False,
    num_workers=cfg.get("data", {}).get("num_workers", 0),
)
print(f"Use IMU: {use_imu}")

In [3]:
# Cell 3: 获取预测结果
@torch.no_grad()
def get_predictions(model, dataloader, device, use_imu, max_samples=10):
    preds, targets = [], []
    for batch in dataloader:
        if use_imu:
            video, imu, ecg = batch
            pred = model(video.to(device), imu.to(device)).cpu().numpy()
        else:
            video, ecg = batch
            pred = model(video.to(device)).cpu().numpy()
        preds.append(pred)
        targets.append(ecg.numpy())
        if sum(len(p) for p in preds) >= max_samples:
            break
    return np.concatenate(preds)[:max_samples], np.concatenate(targets)[:max_samples]

preds, targets = get_predictions(model, test_loader, device, use_imu, N_SAMPLES)
print(f"Shape: preds={preds.shape}, targets={targets.shape}")


Shape: preds=(6, 2500), targets=(6, 2500)


In [ ]:
# Cell 4: 单样本对比（修改 idx 查看不同样本）
idx = 0  # <-- 改这里切换样本
fs = 250
t = np.arange(preds.shape[1]) / fs

pred, target = preds[idx], targets[idx]
r = np.corrcoef(pred, target)[0, 1]
rmse = np.sqrt(np.mean((pred - target) ** 2))

fig, axes = plt.subplots(3, 1, figsize=(14, 7), sharex=True)
axes[0].plot(t, target, 'b', lw=0.8)
axes[0].set_ylabel('Ground Truth')
axes[0].set_title(f'Sample {idx} | RMSE={rmse:.4f} | Pearson r={r:.4f}')

axes[1].plot(t, pred, 'r', lw=0.8)
axes[1].set_ylabel('Predicted')

axes[2].plot(t, target, 'b', lw=0.8, alpha=0.7, label='GT')
axes[2].plot(t, pred, 'r', lw=0.8, alpha=0.7, label='Pred')
axes[2].set_ylabel('Overlay')
axes[2].set_xlabel('Time (s)')
axes[2].legend()

for ax in axes:
    ax.grid(True, alpha=0.3)
plt.tight_layout()
_save = f"{SAVE_DIR}/sample_single.png"
plt.savefig(_save, dpi=120, bbox_inches="tight")
print("Saved:", os.path.abspath(_save))
plt.show()

In [ ]:
# Cell 5: 放大查看细节（修改 start_sec 查看不同时间段）
start_sec = 2  # <-- 改这里
duration = 3

start, end = int(start_sec * fs), int((start_sec + duration) * fs)
t_zoom = t[start:end]

plt.figure(figsize=(14, 4))
plt.plot(t_zoom, target[start:end], 'b', lw=1.2, label='Ground Truth')
plt.plot(t_zoom, pred[start:end], 'r--', lw=1.2, label='Predicted')
plt.xlabel('Time (s)')
plt.ylabel('ECG')
plt.title(f'Sample {idx}: Zoomed {start_sec}-{start_sec+duration}s')
plt.legend()
plt.grid(True, alpha=0.3)
_save = f"{SAVE_DIR}/sample_zoom.png"
plt.savefig(_save, dpi=120, bbox_inches="tight")
print("Saved:", os.path.abspath(_save))
plt.show()

In [ ]:
# Cell 6: 所有样本概览
fig, axes = plt.subplots(N_SAMPLES, 1, figsize=(14, 2.5 * N_SAMPLES), sharex=True)
for i in range(N_SAMPLES):
    r = np.corrcoef(preds[i], targets[i])[0, 1]
    axes[i].plot(t, targets[i], 'b', lw=0.6, alpha=0.7, label='GT')
    axes[i].plot(t, preds[i], 'r', lw=0.6, alpha=0.7, label='Pred')
    axes[i].set_ylabel(f'#{i}')
    axes[i].set_title(f'r={r:.3f}', loc='right', fontsize=10)
    axes[i].grid(True, alpha=0.3)
    if i == 0:
        axes[i].legend(loc='upper right')
axes[-1].set_xlabel('Time (s)')
plt.suptitle(f'All Samples Overview ({SCHEME_NAME})')
plt.tight_layout()
_save = f"{SAVE_DIR}/all_samples.png"
plt.savefig(_save, dpi=120, bbox_inches="tight")
print("Saved:", os.path.abspath(_save))
plt.show()

In [7]:
# Cell 7: 汇总指标
print("Per-sample metrics:")
print("-" * 40)
for i in range(N_SAMPLES):
    r = np.corrcoef(preds[i], targets[i])[0, 1]
    rmse = np.sqrt(np.mean((preds[i] - targets[i]) ** 2))
    mae = np.mean(np.abs(preds[i] - targets[i]))
    print(f"Sample {i}: RMSE={rmse:.4f}, MAE={mae:.4f}, r={r:.4f}")

print("-" * 40)
print(f"Mean RMSE: {np.mean([np.sqrt(np.mean((preds[i]-targets[i])**2)) for i in range(N_SAMPLES)]):.4f}")
print(f"Mean r:    {np.mean([np.corrcoef(preds[i], targets[i])[0,1] for i in range(N_SAMPLES)]):.4f}")


Per-sample metrics:
----------------------------------------
Sample 0: RMSE=0.9727, MAE=0.5632, r=0.0144
Sample 1: RMSE=0.9865, MAE=0.5728, r=-0.1462
Sample 2: RMSE=0.9949, MAE=0.5833, r=-0.2011
Sample 3: RMSE=0.9976, MAE=0.5885, r=-0.1713
Sample 4: RMSE=1.0252, MAE=0.5900, r=-0.1200
Sample 5: RMSE=1.0412, MAE=0.6001, r=-0.1628
----------------------------------------
Mean RMSE: 1.0030
Mean r:    -0.1311


In [ ]:
# Cell 8: 分析视频RGB信号与ECG心率的关系
# 检查视频压缩是否导致PPG信息丢失

import cv2
from scipy import signal as scipy_signal
from scipy.stats import pearsonr
import json

def get_video_info(video_path):
    """获取视频信息"""
    cap = cv2.VideoCapture(video_path)
    info = {
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    }
    info["file_size_mb"] = os.path.getsize(video_path) / (1024 * 1024)
    info["duration_s"] = info["frame_count"] / info["fps"] if info["fps"] > 0 else 0
    info["bitrate_kbps"] = (info["file_size_mb"] * 8 * 1024) / info["duration_s"] if info["duration_s"] > 0 else 0
    cap.release()
    return info

def extract_rgb_signal(video_path, max_frames=None):
    """从视频提取RGB通道均值"""
    cap = cv2.VideoCapture(video_path)
    if max_frames is None:
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    rgb_signals = []
    for _ in range(max_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # BGR -> RGB
        rgb_signals.append([frame[:,:,2].mean(), frame[:,:,1].mean(), frame[:,:,0].mean()])
    cap.release()
    return np.array(rgb_signals)

def bandpass_filter(sig, fs, lowcut=0.7, highcut=4.0, order=3):
    """带通滤波 (0.7-4Hz = 42-240 BPM)"""
    nyq = 0.5 * fs
    b, a = scipy_signal.butter(order, [lowcut/nyq, highcut/nyq], btype='band')
    return scipy_signal.filtfilt(b, a, sig)

def analyze_periodicity(sig, fs):
    """FFT分析心率"""
    sig = sig - sig.mean()
    freqs = np.fft.rfftfreq(len(sig), 1/fs)
    fft_mag = np.abs(np.fft.rfft(sig))
    
    # 心率范围 0.7-3.5 Hz = 42-210 BPM
    hr_mask = (freqs >= 0.7) & (freqs <= 3.5)
    if not hr_mask.any():
        return None, 0
    
    hr_freqs = freqs[hr_mask]
    hr_mags = fft_mag[hr_mask]
    peak_idx = np.argmax(hr_mags)
    dominant_hr = hr_freqs[peak_idx] * 60
    
    # SNR
    snr = hr_mags[peak_idx]**2 / (hr_mags**2).sum()
    return dominant_hr, snr

# 分析前5个样本
samples_dir = "training_data/samples"
pairs = sorted([d for d in os.listdir(samples_dir) if d.startswith("pair_")])[:5]

print("="*70)
print("视频PPG信号与ECG心率关系分析")
print("="*70)

fig, axes = plt.subplots(len(pairs), 4, figsize=(16, 3*len(pairs)))
results = []

for idx, pair in enumerate(pairs):
    pair_dir = os.path.join(samples_dir, pair)
    video_path = os.path.join(pair_dir, "video_0.mp4")
    ecg_path = os.path.join(pair_dir, "ecg.csv")
    meta_path = os.path.join(pair_dir, "metadata.json")
    
    # 视频信息
    vinfo = get_video_info(video_path)
    fps = vinfo['fps']
    
    print(f"\n{pair}: {vinfo['file_size_mb']:.1f}MB, {vinfo['bitrate_kbps']:.0f}kbps, {vinfo['width']}x{vinfo['height']}")
    
    # 提取RGB信号 (10秒)
    max_frames = min(int(10 * fps), vinfo['frame_count'])
    rgb = extract_rgb_signal(video_path, max_frames)
    t_video = np.arange(len(rgb)) / fps
    
    # 红色通道滤波
    red_norm = (rgb[:,0] - rgb[:,0].mean()) / (rgb[:,0].std() + 1e-8)
    red_filt = bandpass_filter(red_norm, fps, 0.7, 4.0)
    
    # PPG心率
    ppg_hr, snr = analyze_periodicity(red_filt, fps)
    
    # ECG
    ecg_df = pd.read_csv(ecg_path)
    ecg = ecg_df["ecg_counts_filt_monitor"].values[:2500]
    t_ecg = np.arange(len(ecg)) / 250
    
    # 标注心率
    with open(meta_path) as f:
        meta = json.load(f)
    annotated_hr = meta.get("heart_rate")
    
    if ppg_hr:
        error = abs(ppg_hr - annotated_hr)
        status = "✓" if error < 10 else ("△" if error < 20 else "✗")
        print(f"  PPG心率: {ppg_hr:.1f} BPM, 标注: {annotated_hr} BPM, 误差: {error:.1f} {status}")
    else:
        print(f"  PPG心率: 未检测到, 标注: {annotated_hr} BPM")
    print(f"  信号SNR: {snr:.3f}, RGB std: R={rgb[:,0].std():.2f}, G={rgb[:,1].std():.2f}, B={rgb[:,2].std():.2f}")
    
    results.append({'pair': pair, 'ppg_hr': ppg_hr, 'annotated_hr': annotated_hr, 'snr': snr, 'bitrate': vinfo['bitrate_kbps']})
    
    # 绘图
    ax = axes[idx] if len(pairs) > 1 else axes
    
    # 红色原始
    ax[0].plot(t_video, rgb[:,0], 'r-', lw=0.5)
    ax[0].set_ylabel(f'{pair}\nRed')
    if idx == 0: ax[0].set_title('Red Channel (Raw)')
    ax[0].grid(True, alpha=0.3)
    
    # 红色滤波
    ax[1].plot(t_video, red_filt, 'r-', lw=0.5)
    ax[1].set_ylabel('Red Filt')
    if idx == 0: ax[1].set_title('Red (0.7-4Hz BP)')
    ax[1].grid(True, alpha=0.3)
    
    # FFT
    freqs = np.fft.rfftfreq(len(red_filt), 1/fps)
    fft_mag = np.abs(np.fft.rfft(red_filt))
    hr_mask = (freqs >= 0.5) & (freqs <= 4)
    ax[2].plot(freqs[hr_mask]*60, fft_mag[hr_mask], 'b-')
    if ppg_hr: ax[2].axvline(ppg_hr, color='r', linestyle='--', alpha=0.7, label=f'PPG: {ppg_hr:.0f}')
    if annotated_hr: ax[2].axvline(annotated_hr, color='g', linestyle=':', alpha=0.7, label=f'GT: {annotated_hr}')
    ax[2].set_xlim(40, 150)
    ax[2].legend(fontsize=8)
    if idx == 0: ax[2].set_title('FFT (BPM)')
    ax[2].grid(True, alpha=0.3)
    
    # ECG
    ecg_norm = (ecg - ecg.mean()) / (ecg.std() + 1e-8)
    ax[3].plot(t_ecg, ecg_norm, 'b-', lw=0.5)
    if idx == 0: ax[3].set_title('ECG')
    ax[3].grid(True, alpha=0.3)
    
    if idx == len(pairs) - 1:
        for a in ax: a.set_xlabel('Time (s)')

plt.tight_layout()
_save = f"{SAVE_DIR}/ppg_ecg_analysis.png"
plt.savefig(_save, dpi=150, bbox_inches='tight')
print(f"\n图片已保存: {_save}")
plt.show()

# 汇总
print("\n" + "="*70)
print("汇总")
print("="*70)
valid = [r for r in results if r['ppg_hr']]
if valid:
    errors = [abs(r['ppg_hr'] - r['annotated_hr']) for r in valid]
    print(f"有效PPG心率检测: {len(valid)}/{len(results)}")
    print(f"PPG-ECG心率误差: 平均={np.mean(errors):.1f} BPM")
    print(f"平均码率: {np.mean([r['bitrate'] for r in results]):.0f} kbps")
    print(f"平均SNR: {np.mean([r['snr'] for r in results]):.3f}")